<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/Binary_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

In [2]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols


#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

def get_crypto_syms():
   screens = [
    'all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
      return symbols

250


15

In [4]:
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
       fixed_df = []
       df = np.array(df)
       for i in range(20, df.shape[0]-1):
           twenty_days_ago = i-20
           ninteen_days_ago = i-19
           eighteen_days_ago = i-18
           seventeen_days_ago = i-17
           sixteen_days_ago = i-16
           fifteen_days_ago = i-15
           fourteen_days_ago = i-14
           thirteen_days_ago = i-13
           twelve_days_ago = i-12
           eleven_days_ago = i-11
           ten_days_ago = i-10
           nine_days_ago = i-9
           eight_days_ago = i-8
           seven_days_ago = i-7
           six_days_ago = i-6
           five_days_ago = i-5
           four_days_ago = i-4
           three_days_ago = i-3
           two_days_ago = i - 2
           yesterday = i - 1
           today = i
           tomorrow = i + 1
           if df[tomorrow][3] > df[today][3]:
               future = 1
           else:
               future = 0


           row = [df[twenty_days_ago][3],df[ninteen_days_ago][3],df[eighteen_days_ago][3],df[seventeen_days_ago][3],df[sixteen_days_ago][3],df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],df[twelve_days_ago][3],df[eleven_days_ago][3],
                   df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                   df[today][3], future]

           fixed_df.append(row)


       arrayed_fixed_df = np.array([fixed_df])
       two_d_fixed_df = []
       for i in range(0, arrayed_fixed_df.shape[1]):
          two_d_fixed_df.append(arrayed_fixed_df[0][i])
    
       mm = np.array(two_d_fixed_df)


       column_names = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago","seventeen_days_ago","sixteen_days_ago","fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today", "result"]
    
       column_names_without_result = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago","seventeen_days_ago","sixteen_days_ago","fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today"]
       dff = pd.DataFrame(mm, columns=column_names)
       scaler = MinMaxScaler()
       result = dff["result"]
       dff = dff.drop(["result"],axis=1)
       scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=column_names_without_result,dtype=object)

       scaled["result"] = result
       
       fixed_dfs.append(scaled)

   data = fixed_dfs[0]
   for i in range(1,len(dfs)):
     data = pd.concat([data,fixed_dfs[i]], ignore_index = True)
   data = data.astype(float)
   data = data.dropna()
   ttext = getsizeof(data)
   data.to_parquet(f"df_{ttext}.parquet")
   return data
   
def spliting(data):
  X = data.drop(["result"],axis=1)
  y = data["result"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest

In [5]:
dfs = []
symbols = get_crypto_syms()
for i in symbols:
      data = yf.download(i,period="730d",interval="1h",progress=False)
      if data.empty :
        print("Passing...")
      else:
          dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
       if df.shape[0] > 21:
          dfs.append(df)
data = process(dfs)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(1426969, 21) (1426969,)
(158553, 21) (158553,)


,twenty_days_ago,ninteen_days_ago,eighteen_days_ago,seventeen_days_ago,sixteen_days_ago,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,...,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,0.205922,0.169698,0.018891,0.000000,0.011831,0.066732,0.032731,0.056153,0.072333,0.115331,...,0.276723,0.363487,0.561166,0.622983,0.891798,0.876681,0.823477,1.000000,0.937301,1.0
1,0.169698,0.018891,0.000000,0.011831,0.066732,0.032731,0.056153,0.072333,0.115331,0.090036,...,0.363487,0.561166,0.622983,0.891798,0.876681,0.823477,1.000000,0.937301,0.960334,0.0
2,0.018891,0.000000,0.011831,0.066732,0.032731,0.056153,0.072333,0.115331,0.090036,0.135038,...,0.561166,0.622983,0.891798,0.876681,0.823477,1.000000,0.937301,0.960334,0.701791,0.0
3,0.217252,0.226512,0.269486,0.242872,0.261206,0.273870,0.307527,0.287728,0.322953,0.433856,...,0.704891,0.915305,0.903472,0.861827,1.000000,0.950923,0.968952,0.766577,0.000000,0.0
4,0.343238,0.379726,0.357129,0.372696,0.383449,0.412026,0.395215,0.425125,0.519292,0.576957,...,0.928087,0.918039,0.882679,1.000000,0.958329,0.973637,0.801803,0.150908,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585517,0.767251,0.710036,0.660409,0.627730,0.653846,0.629789,0.491728,0.616451,0.641720,0.698879,...,1.000000,0.937299,0.895154,0.520683,0.505828,0.296428,0.181631,0.000000,0.088929,0.0
1585518,0.710036,0.660409,0.627730,0.653846,0.629789,0.491728,0.616451,0.641720,0.698879,0.894939,...,0.937299,0.895154,0.520683,0.505828,0.296428,0.181631,0.000000,0.088929,0.024433,1.0
1585519,0.660409,0.627730,0.653846,0.629789,0.491728,0.616451,0.641720,0.698879,0.894939,0.946322,...,0.895154,0.520683,0.505828,0.296428,0.181631,0.000000,0.088929,0.024433,0.125319,0.0
1585520,0.635676,0.661236,0.637692,0.502578,0.624639,0.649368,0.705307,0.897182,0.947467,1.000000,...,0.530915,0.516377,0.311447,0.199101,0.021347,0.108377,0.045258,0.143991,0.000000,0.0


In [31]:
#for symbols in new_arrays:
#   name = f"df_{symbols[0]}.parquet"
#   dfs = []
#   for i in symbols:
#      data = yf.download(i,period="730d",interval="1h")
#      if data.empty :
#        print("Passing...")
#      else:
#          dfs.append(data)
#   ndfs = dfs
#   dfs = []
#   for df in ndfs:
#       if df.shape[0] > 21:
#          dfs.append(df)
#   process(dfs,name)

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

Random Forest :  0.4971751412429379
Logistic Regression: 0.5819209039548022
Nearest Centroid: 0.5141242937853108
GaussianNB: 0.5310734463276836
K-Neighbors: 0.5254237288135594
Decision Tree: 0.4689265536723164
Gradient Boost: 0.536723163841808
SVM: 0.5310734463276836


In [13]:
model = Sequential()
model.add(Dense(3000, activation='relu', kernel_initializer='he_normal', input_shape=(xTrain.shape[1],)))
model.add(Dense(1750, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1000, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 2600)              57200     
                                                                 
 dense_21 (Dense)            (None, 1750)              4551750   
                                                                 
 dense_22 (Dense)            (None, 1250)              2188750   
                                                                 
 dense_23 (Dense)            (None, 500)               625500    
                                                                 
 dense_24 (Dense)            (None, 1)                 501       
                                                                 
Total params: 7,423,701
Trainable params: 7,423,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=11,batch_size=100,validation_data=(xTest,yTest))

Epoch 1/11
 9350/14270 [==================>...........] - ETA: 13:24 - loss: 0.6929 - accuracy: 0.5234

In [ ]:
model.evaluate(xTest,yTest)

In [ ]:
model.save("weights_91_1500_daily_crypto.h5")